In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from typing import Dict, List
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc,roc_auc_score 
from sklearn.decomposition import PCA
from functools import reduce
from scipy.io import loadmat
import cv2
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.ops as ops
torch.cuda.set_device(7)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


7


In [2]:
'''
#Code: https://github.com/imatge-upc/retrieval-2016-deepvision
       https://github.com/WXinlong/SOLO/blob/master/mmdet/models/detectors/faster_rcnn.py 
#Ref:
       https://github.com/jwyang/faster-rcnn.pytorch
       https://github.com/Liu-Yicheng/Faster-rcnn
       R-FCN：https://github.com/princewang1994/RFCN_CoupleNet.pytorch
       YOLOv3：https://github.com/eriklindernoren/PyTorch-YOLOv3
       SSD：https://github.com/amdegroot/ssd.pytorch
#Paper: CVPR2016《Faster r-cnn features for instance search》
'''
import sys;
sys.path.append("/data/tmpexec/VOC2007/SOLO")
from mmdet.apis import init_detector, inference_detector, show_result_pyplot, show_result_ins
import mmcv

def pool_feats(feat, pooling='max'):
    feat = np.expand_dims(feat,axis=0)
    if pooling is 'max':
        feat = np.max(np.max(feat,axis=2),axis=1)
    else:
        feat = np.sum(np.sum(feat,axis=2),axis=1)
    return feat

config_file = '/data/tmpexec/VOC2007/SOLO/configs/solo/decoupled_solo_r50_fpn_8gpu_3x.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint_file = '/data/tmpexec/VOC2007/SOLO/checkpoints/DECOUPLED_SOLO_R50_3x.pth'
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

tstart = time.time()
root_dir = '/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/' #the path of images
trData = pd.read_csv(root_dir+"trainset.csv" , sep=',')
teData = pd.read_csv(root_dir+"testset.csv" , sep=',')
#trainset 
trN, trM, trY, trF = [],[],[],[]
for iname, itype in np.array(trData).tolist():
    try:
        trN.append(iname)
        trY.append(itype) #0 refer to Benign, and 1 refers to malignant
        image_path = os.path.join(root_dir, 'image', iname)
        result = inference_detector(model, image_path)
        result = result[0][0].cpu().data.numpy()
        feat = pool_feats(result)
        trF.append(feat.flatten()[0:128])
        mask_path = os.path.join(root_dir,'mask', iname)
        mask = cv2.resize(cv2.imread(mask_path).astype(np.float32), (256, 256))#(256,256)
        trM.append(mask)
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(trN),trData.shape[0]))
    sys.stdout.flush()
print('The length of trainset is %d'%len(trN))
#testset
teN, teM, teY, teF = [],[],[],[]
for iname, itype in np.array(teData).tolist():
    try:
        teN.append(iname)
        teY.append(itype) #0 refer to Benign, and 1 refers to malignant
        image_path = os.path.join(root_dir, 'image', iname)
        result = inference_detector(model, image_path)
        result = result[0][0].cpu().data.numpy()
        feat = pool_feats(result)
        teF.append(feat.flatten()[0:128])
        mask_path = os.path.join(root_dir,'mask', iname)
        mask = cv2.resize(cv2.imread(mask_path).astype(np.float32), (256, 256))#(256,256)
        teM.append(mask)
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(teN),teData.shape[0]))
    sys.stdout.flush()
print('The length of testset is %d'%len(teN))

elapsed = time.time() - tstart
print('Completed data handle in %d seconds' % int(elapsed))

1 / 3279 

/root/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py:2539: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


120 / 3279 1216.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1216.PNG
264 / 3279 4662.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/4662.PNG
398 / 3279 1309.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1309.PNG
400 / 3279 1473.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1473.PNG
448 / 3279 4501.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/4501.PNG
524 / 3279 2558.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/2558.PNG
532 / 3279 4432.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/4432.PNG
582 / 3279 53.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/53.PNG
770 / 3279 1786.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1786.PNG
814 / 3279 1306.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1306.PNG
994 / 3279 631.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/631.PNG
1227 / 3279 4596.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/4596.PNG
1347 / 3279 1339.PNG:/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1339.PNG
1402 / 3279 4624

In [3]:
print(np.array(trF).shape)
print(np.array(teF).shape)
print(feat.shape)

(3258, 128)
(361, 128)
(1, 856)


In [ ]:
#compute the size of lesion
def Func_IOU_size(pred,target):
    ious = []
    # ignore IOU for background class
    pred_inds = pred != 0
    pred_sum = pred_inds.sum()
    target_inds = target != 0
    target_sum = target_inds.sum()
    ious.append(round(float(min(pred_sum,target_sum)/max(pred_sum,target_sum)),4))
    return np.mean(ious)
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(128) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5, 10, 20, 50]:
    mAP = [] #mean average precision
    mIoU = []
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                pos_len = pos_len +1
                mAP.append(pos_len/rank_len) 
            else: 
                mAP.append(0)
            mIoU.append(Func_IOU_size(teM[i],trM[j]))
    print("mAP={:.4f}, mIoU={:.4f}".format(np.mean(mAP),np.mean(mIoU)))

Completed buliding index in 21 seconds
mAP=0.3996, mIoU=0.3935
mAP=0.3658, mIoU=0.4013
mAP=0.3555, mIoU=0.3925


In [38]:
#https://hellozhaozheng.github.io/z_post/PyTorch-YOLO/
from __future__ import division
import sys;
sys.path.append("/data/tmpexec/PyTorch-YOLOv3")

from models import *
from utils.utils import *
from utils.datasets import *

import os
import sys
import time
import datetime
import argparse

from PIL import Image

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

import argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--image_folder", type=str, default="/data/tmpexec/PyTorch-YOLOv3/data/samples", help="path to dataset")
    parser.add_argument("--model_def", type=str, default="/data/tmpexec/PyTorch-YOLOv3/config/yolov3.cfg", help="path to model definition file")
    parser.add_argument("--weights_path", type=str, default="/data/tmpexec/PyTorch-YOLOv3/weights/yolov3.weights", help="path to weights file")
    parser.add_argument("--class_path", type=str, default="/data/tmpexec/PyTorch-YOLOv3/data/coco.names", help="path to class label file")
    parser.add_argument("--conf_thres", type=float, default=0.8, help="object confidence threshold")
    parser.add_argument("--nms_thres", type=float, default=0.4, help="iou thresshold for non-maximum suppression")
    parser.add_argument("--batch_size", type=int, default=1, help="size of the batches")
    parser.add_argument("--n_cpu", type=int, default=0, help="number of cpu threads to use during batch generation")
    parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
    parser.add_argument("--checkpoint_model", type=str, help="path to checkpoint model")
    opt = parser.parse_args(args=[])
    #opt = parser.parse_args()
    print(opt)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #os.makedirs("output", exist_ok=True)

    # Set up model
    model = Darknet(opt.model_def, img_size=opt.img_size).to(device)

    if opt.weights_path.endswith(".weights"):
        # Load darknet weights
        model.load_darknet_weights(opt.weights_path)
    else:
        # Load checkpoint weights
        model.load_state_dict(torch.load(opt.weights_path))

    model.eval()  # Set in evaluation mode
    
    dataloader = DataLoader(
        ImageFolder(opt.image_folder, img_size=opt.img_size),
        batch_size=opt.batch_size,
        shuffle=False,
        num_workers=opt.n_cpu,
    )

    classes = load_classes(opt.class_path)  # Extracts class labels from file

    Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    imgs = []  # Stores image paths
    img_detections = []  # Stores detections for each image index

    print("\nPerforming object detection:")
    prev_time = time.time()
    for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
        # Configure input
        input_imgs = Variable(input_imgs.type(Tensor))
        print(input_imgs.shape)

        # Get detections
        with torch.no_grad():
            detections = model(input_imgs)
            #fdfd = pool_feats(detections)
            #print(detections.shape)
             # detections的shape为: [1,10647,85], 其中, 1为batch_size
            # 因为对于尺寸为416的图片来说:(13*13+26*26+52*52) * 3 = 10647
            # 如果图片尺寸为608(必须为32的倍数), 那么就为:(19*19+38*38+76*76) * 3 = 22743
            detections = non_max_suppression(detections, opt.conf_thres, opt.nms_thres)

        # Log progress
        current_time = time.time()
        inference_time = datetime.timedelta(seconds=current_time - prev_time)
        prev_time = current_time
        print("\t+ Batch %d, Inference Time: %s" % (batch_i, inference_time))

        # Save image and detections
        imgs.extend(img_paths)
        img_detections.extend(detections)

    # Bounding-box colors
    cmap = plt.get_cmap("tab20b")
    colors = [cmap(i) for i in np.linspace(0, 1, 20)]

    print("\nSaving images:")
    # Iterate through images and save plot of detections
    for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):

        print("(%d) Image: '%s'" % (img_i, path))

        # Create plot
        img = np.array(Image.open(path))
        plt.figure()
        fig, ax = plt.subplots(1)
        ax.imshow(img)

        # Draw bounding boxes and labels of detections
        if detections is not None:
            # Rescale boxes to original image
            detections = rescale_boxes(detections, opt.img_size, img.shape[:2])
            unique_labels = detections[:, -1].cpu().unique()
            n_cls_preds = len(unique_labels)
            bbox_colors = random.sample(colors, n_cls_preds)
            for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

                print("\t+ Label: %s, Conf: %.5f" % (classes[int(cls_pred)], cls_conf.item()))

                box_w = x2 - x1
                box_h = y2 - y1

                color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
                # Create a Rectangle patch
                bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor="none")
                # Add the bbox to the plot
                ax.add_patch(bbox)
                # Add label
                plt.text(
                    x1,
                    y1,
                    s=classes[int(cls_pred)],
                    color="white",
                    verticalalignment="top",
                    bbox={"color": color, "pad": 0},
                )

        # Save generated image with detections
        plt.axis("off")
        plt.gca().xaxis.set_major_locator(NullLocator())
        plt.gca().yaxis.set_major_locator(NullLocator())
        filename = path.split("/")[-1].split(".")[0]
        plt.savefig(f"/data/tmpexec/PyTorch-YOLOv3/output/{filename}.png", bbox_inches="tight", pad_inches=0.0)
        plt.close()

Namespace(batch_size=1, checkpoint_model=None, class_path='/data/tmpexec/PyTorch-YOLOv3/data/coco.names', conf_thres=0.8, image_folder='/data/tmpexec/PyTorch-YOLOv3/data/samples', img_size=416, model_def='/data/tmpexec/PyTorch-YOLOv3/config/yolov3.cfg', n_cpu=0, nms_thres=0.4, weights_path='/data/tmpexec/PyTorch-YOLOv3/weights/yolov3.weights')

Performing object detection:
torch.Size([1, 3, 416, 416])
torch.Size([1, 10647, 85])
	+ Batch 0, Inference Time: 0:00:00.053645
torch.Size([1, 3, 416, 416])
torch.Size([1, 10647, 85])
	+ Batch 1, Inference Time: 0:00:00.027473
torch.Size([1, 3, 416, 416])
torch.Size([1, 10647, 85])
	+ Batch 2, Inference Time: 0:00:00.025320
torch.Size([1, 3, 416, 416])
torch.Size([1, 10647, 85])
	+ Batch 3, Inference Time: 0:00:00.131455
torch.Size([1, 3, 416, 416])
torch.Size([1, 10647, 85])
	+ Batch 4, Inference Time: 0:00:00.030438
torch.Size([1, 3, 416, 416])
torch.Size([1, 10647, 85])
	+ Batch 5, Inference Time: 0:00:00.027984
torch.Size([1, 3, 416, 416])
t

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [11]:
import sys;
sys.path.append("/data/tmpexec/VOC2007/SOLO")
from mmdet.apis import init_detector, inference_detector, show_result_pyplot, show_result_ins
import mmcv


config_file = '/data/tmpexec/VOC2007/SOLO/configs/solo/decoupled_solo_r50_fpn_8gpu_3x.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint_file = '/data/tmpexec/VOC2007/SOLO/checkpoints/DECOUPLED_SOLO_R50_3x.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image
img = '/data/fjsdata/MCBIR-Ins/TNSCUI2020_train/image/1216.PNG'
result = inference_detector(model, img)


#show_result_ins(img, result, model.CLASSES, score_thr=0.25, out_file="/data/tmpexec/VOC2007/SOLO/checkpoints/demo_out.jpg")

In [10]:
print(result[0][0].shape)
print(result[0][1].shape)
print(result[0][2].shape)
def pool_feats(feat, pooling='max'):
    if pooling is 'max':
        feat = np.max(np.max(feat,axis=2),axis=1)
    else:
        feat = np.sum(np.sum(feat,axis=2),axis=1)
    return feat

feat = np.expand_dims(result[0][0].cpu(),axis=0)
print(pool_feats(feat).flatten().shape)

TypeError: 'NoneType' object is not subscriptable

In [81]:
img = cv2.resize(cv2.imread( '/data/tmpexec/VOC2007/SOLO/checkpoints/street.jpg').astype(np.float32), (1333, 800))
img = np.expand_dims(img,axis=0)
img = torch.from_numpy(img).type(torch.FloatTensor).cuda()
feat = model.extract_feat(img.permute(0, 3, 1, 2))
print(feat.shape)

RuntimeError: The size of tensor a (167) must match the size of tensor b (168) at non-singleton dimension 3